In [ ]:
import zipfile
zip_file = zipfile.ZipFile('word2vec-nlp-tutorial.zip', 'r')
zip_file.extractall()
zip_file.close()

In [ ]:
import pandas as pd

# Файл labeledTrainData.tsv содержит тексты, которые размеченны по классам
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)

# Файл testData.tsv содержит тексты, по которым нужно выдать предсказания
test = pd.read_csv('testData.tsv', header=0,  delimiter="\t", quoting=3)

# Обратим внимание на unlabeledTrainData.tsv
# Для данного файла нет меток, к какому классу относятся его тексты
# Так же организаторы не ждут предсказания для по классам для текстов из данного файла
# В нем представленны тексты того же посола, что и остальные
# А значит добавив его в обучение word2vec мы улучшим знание нашемй модели о "мире"
unsup = pd.read_csv('unlabeledTrainData.tsv', header=0,  delimiter="\t", quoting=3)

Напишем функцию которая для представленного текста:
* меняет n't на not (по желанию)
* приводит к нижнему регистру
* делит на слова
* удаляет стоп-слова (по желанию)

In [ ]:
from bs4 import BeautifulSoup
import re
#from nltk.corpus import stopwords

def text_to_wordlist(text):
    text = re.sub('n\'t', ' not', text)
    
    text = re.sub('[^a-zA-Z]', ' ', text)
    words = text.lower().split()

    #stops = set(stopwords.words("english"))

    return words

In [ ]:
# Протестируем данную функцию

text_to_wordlist("""
Tyrone Garland <s>(born 1992)</s> is an American professional basketball player 
who last played with the National <b>Basketball</b> League of Canada's Mississauga Power
""")

# Посмотрим что результатом будет list, элементы которого - слова

Напишем функцию которая для представленного текста:
* удаляет html теги
* производит деления на предложения
* каждое предложение делит на слова (применяя выше написанную функцию)

In [ ]:
import nltk.data

def text_to_sentences(text):
    text = BeautifulSoup(text).get_text()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))

    return sentences

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Протестируем данную функцию

text_to_sentences("""
Sherlock Holmes is a four-act play written by <p/> William Gillette and Sir Arthur Conan Doyle, 
based on Conan Doyle's eponymous character. It drew material from the stories 
<s>"A Scandal in Bohemia"</s>, "The Final Problem", and A Study in Scarlet, pitting Holmes 
against Professor Moriarty and reinventing the character of Irene Adler as a new love 
interest named Alice Faulkner. This play introduced the phrase "Elementary, my dear Watson" 
and Holmes' curved pipe.
""")


# Посмотрим что результатом будет list, элементы которого - list, элементы которого - слова
# т.е. получаем list list'ов (список списков)

Так давайте соберем все имеющиеся тексты в подобню структуру

In [ ]:
%%time
sentences = []

for review in train["review"]:
    sentences += text_to_sentences(review)

In [ ]:
%%time
for review in unsup["review"]:
    sentences += text_to_sentences(review)

In [ ]:
%%time
for review in test["review"]:
    sentences += text_to_sentences(review)

In [ ]:
# В конце сбора sentences будет list list'ов (список списков) - как и пример выше.
# (Повторюсь) каждый элемент списка sentences - предложение, но представленное в виде списка слов - потому список

# выведем количество элементов этого массива (оно же - количество предложений во всех текста)
print(len(sentences))

In [ ]:
# а так же посмотрим на сам массив
print ('\t Первый элемент массива')
print (sentences[0]) 
print ('\t Второй элемент массива')
print (sentences[1])

Обучим же теперь модель Word2Vec

In [ ]:
import logging 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
log = logging.getLogger()
log.setLevel(logging.INFO)

In [ ]:
%%time
from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
params = {}
params['size'] = 300  # итоговая размерность вектора каждого слова
params['min_count'] = 5  # минимальная частотность слова, чтобы оно попало в модель
params['workers'] = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
params['window'] = 10        # размер окна 
params['sample'] = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, **params)

In [ ]:
# Финализируем нашу модель. Ее нельзя будет доучить теперь, но она начнет занимать гораздо меньше места
model.init_sims(replace=True)

Натренировав модель, получили представление каждого слова в семантическом пространстве (часто называют "псевдо" семантическое пространство)

Попробуем популярный пример: QUEEN + MAN - KING = ? 

In [ ]:
model.most_similar(positive=['queen', 'man'], negative=['king'])[0]

In [ ]:
# Посмотрим слова, которые очень похожи на слово MOVIE

model.most_similar('movie')

# Попробуем и другие слова

In [ ]:
model.most_similar('awful')

In [ ]:
model.most_similar('big')

In [ ]:
model.most_similar('mail')

In [ ]:
# так же взглянем на функцию doesnt_match, она покажет лишнее слово в массив

model.doesnt_match(['berlin', 'rome', 'africa', 'london', 'paris'])

In [ ]:
model.doesnt_match(['man', 'woman', 'child', 'dog'])

In [ ]:
# давайте просмотрим вектор одного из слов
# его длину
print (len(model['moon']))

# и сам вектор
print (model['moon'])

In [ ]:
# Словарь - все слова которые участвуют в модели можно просмотреть так
len(model.wv.vocab.keys())

Т.к. мы хотим классифицировать не слова, а тексты, надо перевести тексты в вектора (представить в виде фич)
Один из простых методов - сложить все вектора слов входящих в текст и поделить на число входящих слов.
Напишем функцию, которая:
* создает нулевой вектор - это будет результирующий вектор
* идем по всем словам в тексте, если слово есть в моделе:
  * увеличиваем счетчик слов
  * прибавим вектор слова к результирующему вектору
* поделим все координаты на число слов, вектора которых мы прибавляли к результирующему вектору

In [ ]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.vocab.keys())
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

Напишем функцию, которая на входе получает список всех текстов, а на выходе отдает список вектор каждого текста - что является прямоугольной матрицей

In [ ]:
from tqdm import tqdm_notebook as tqdm
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in tqdm(enumerate(texts),):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

Заметим что функция texts_to_vecs принимает не просто тексты, а список всех слов текста. 

(!!!) Внимание: не список списков (там где сначала делили на предложения, а предложения на слова), а обычный линейный список

Но у нас есть функции, которые переводят 1) текст в список предложений, 2) предложение в список слов

Может показаться, что можно использовать 2ую функцию, но придется тогда ее переписать, потому как теги у нас удаляются лишь в первой функции

In [ ]:
# Поступим иначе, в python есть возможность развернуть двухмерный массив в одномерный, вот пример

temp_list = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]
print (sum(temp_list, []))

# магия :)

In [ ]:
# Мы же такой возможностью воспользуемся, зная что функция text_to_sentences возвращает список списков

list_of_list_of_words = text_to_sentences("""
Sherlock Holmes is a four-act play written by <p/> William Gillette and Sir Arthur Conan Doyle, 
based on Conan Doyle's eponymous character. It drew material from the stories 
<s>"A Scandal in Bohemia"</s>, "The Final Problem", and A Study in Scarlet, pitting Holmes 
against Professor Moriarty and reinventing the character of Irene Adler as a new love 
interest named Alice Faulkner. This play introduced the phrase "Elementary, my dear Watson" 
and Holmes' curved pipe.
""")

print ('\t длина list_of_list_of_words')
print (len(list_of_list_of_words))
print ('\t первый элемент list_of_list_of_words')
print (list_of_list_of_words[0])

list_of_words = sum(list_of_list_of_words, [])

print ('\t длина list_of_words')
print (len(list_of_words))
print ('\t первый элемент list_of_words')
print (list_of_words[0])

In [ ]:
# действительно работает, сделаем для всех текстов из train
train_like_word_list = [sum(text_to_sentences(text), []) for text in train['review']]

In [ ]:
train_vecs = texts_to_vecs(train_like_word_list, model, params['size'])

In [ ]:
# сделаем тоже самое для test
test_like_word_list = [sum(text_to_sentences(text), []) for text in test['review']]

In [ ]:
test_vecs = texts_to_vecs(test_like_word_list, model, params['size'])

In [ ]:
%%time
# Воспользуемся train_vecs, test_vecs, train["sentiment"] 
#    как матрица фичей обучающей выборки, матрица фичей тестовой выборки, таргет для обучающей выборки соответственно

# Стандартный случайный лес в таком случае
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train["sentiment"])
predict = forest.predict(test_vecs)

# И вот задача решена